In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import *
from IPython.display import HTML
# mtx and dist are used for undistort images.

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
print(images)
# Step through the list and search for chessboard corners
shape = ()
for fname in images:
    img = cv2.imread(fname)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    shape=gray.shape
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)


def undistortImage(image):
    undist_image = cv2.undistort(image,mtx,dist,None,mtx)
    return undist_image
    


['camera_cal/calibration10.jpg', 'camera_cal/calibration7.jpg', 'camera_cal/calibration9.jpg', 'camera_cal/calibration2.jpg', 'camera_cal/calibration17.jpg', 'camera_cal/calibration3.jpg', 'camera_cal/calibration1.jpg', 'camera_cal/calibration16.jpg', 'camera_cal/calibration15.jpg', 'camera_cal/calibration4.jpg', 'camera_cal/calibration5.jpg', 'camera_cal/calibration11.jpg', 'camera_cal/calibration8.jpg', 'camera_cal/calibration14.jpg', 'camera_cal/calibration6.jpg', 'camera_cal/calibration19.jpg', 'camera_cal/calibration20.jpg', 'camera_cal/calibration18.jpg', 'camera_cal/calibration13.jpg', 'camera_cal/calibration12.jpg']


In [2]:
def grayScale(image):
    return cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
def abs_sobel_thresh(gray_image,orient='x',ksize=3,thresh=(0,255)):
    if orient == 'x':
        sobelx = cv2.Sobel(gray_image,cv2.CV_64F,1,0)
    else:
        sobelx = cv2.Sobel(gray_image,cv2.CV_64F,0,1)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sxbinary
def mag_thresh(img, ksize=3, thresh=(0, 255)):
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1

    # Return the binary image
    return binary_output
def dir_threshold(img, ksize=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output
def conbinedThresh(image,ksize=5):
    image = grayScale(image)
    gradx = abs_sobel_thresh(image, orient='x', ksize=3, thresh=(10, 70))
#     print(gradx.shape)
    grady = abs_sobel_thresh(image, orient='y', ksize=3, thresh=(10, 70))
#     print(grady.shape)
    mag_binary = mag_thresh(image, ksize=3, thresh=(10, 80))
#     print(mag_binary.shape)
    dir_binary = dir_threshold(image, ksize=15, thresh=(0,0.4))
#     print(dir_binary.shape)
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined

In [3]:

def colorThreshold(image):
#     gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
#     thresh = (150,220) # threshold for Gray
#     binary = np.zeros_like(gray)
#     binary[(gray>thresh[0]) & (gray <= thresh[1])] = 1

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    thresh = (85, 255) # for S threshold
    binarys = np.zeros_like(S)
    binarys[(S > thresh[0]) & (S <= thresh[1])] = 1

    thresh = (15, 150) # for H threshold
    binaryh = np.zeros_like(H)
    binaryh[(H > thresh[0]) & (H <= thresh[1])] = 1
    
    conbined = np.zeros_like(H)
    conbined[(binaryh ==1) & (binarys ==1)] = 1
    return conbined
def conbinedThreshold(img):
    colord = colorThreshold(img)
#     gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    print(gray.shape)
    combine = conbinedThresh(img)
    combined = np.zeros_like(combine)
    combined[(combine == 1) | (colord == 1)] = 1
    return combined

In [4]:
# perspective transform parameters
p1 = "test_images/straight_lines1.jpg"
p1 = "test_images/test1.jpg"
image = mpimg.imread(p1)
image = undistortImage(image)
img_size = (image.shape[1],image.shape[0])
src = np.float32([[570,470],[715,470],[1040,685],[255,685]])
# dst = np.float32([[255,470],[1040,470],[1040,685],[255,685]])
xoffset = 300
yoffset = 0
dst = np.float32([[xoffset,yoffset],[img_size[0]-xoffset,yoffset],
                  [img_size[0]-xoffset,img_size[1]-yoffset],[xoffset,img_size[1]-yoffset]])
M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst,src)

def warpImage(image):
    img_size = image.shape
#     ht_window = np.uint(img_size[0]/1.5)
#     hb_window = np.uint(img_size[0])
#     c_window = np.uint(img_size[1]/2)
#     ctl_window = c_window - .25*np.uint(img_size[1]/2)
#     ctr_window = c_window + .25*np.uint(img_size[1]/2)
#     cbl_window = c_window - 1*np.uint(img_size[1]/2)
#     cbr_window = c_window + 1*np.uint(img_size[1]/2)
#     src = np.float32([[cbl_window,hb_window],[cbr_window,hb_window],
#                       [ctr_window,ht_window],[ctl_window,ht_window]])
#     dst = np.float32([[0,img_size[0]],[img_size[1],img_size[0]],
#                   [img_size[1],0],[0,0]])
#     M = cv2.getPerspectiveTransform(src,dst)
#     Minv = cv2.getPerspectiveTransform(dst,src)
# #     warped,M_warp,Minv_warp = warp_image(image,src,dst,(img_size[1],img_size[0])) # returns birds eye image
# #     return warped,M_warp,Minv_warp
    warped = cv2.warpPerspective(image, M, (img_size[1],img_size[0]))
    return warped

In [5]:
def maskedImage(image,binary_warped):
    
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    print(histogram.shape)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

#     print('out_image shape: ',out_img.shape)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    print(midpoint)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

#     print("leftx_base: ",leftx_base," rightx_base: ",rightx_base)
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []


    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        print((win_xleft_low,win_y_low),(win_xleft_high,win_y_high))
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 10) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 10) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
#     print('leftx:',leftx)
#     print('lefty:',lefty)
#     print('righty:',righty)
#     print('rightx:',rightx)
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    # image = undistortImage(image)
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.5, 0)
    return result

In [6]:
# process image function
# 1. undistort image
# 2. thresholding
# 3. perspective transform
# 4. curved lines
def process_image(image):
    img = undistortImage(image)
    warped_img = warpImage(img)
    threshed = conbinedThreshold(warped_img)
    image = maskedImage(img,threshed)
    return image

In [7]:
#process image from vedio file and write to mp4

white_output = 'LaneFinding.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
ipython_display(white_clip)

(720, 1280)
(1280,)
640
(271, 640) (431, 720)
(277, 560) (437, 640)
(276, 480) (436, 560)
(269, 400) (429, 480)
(265, 320) (425, 400)
(257, 240) (417, 320)
(246, 160) (406, 240)
(237, 80) (397, 160)
(224, 0) (384, 80)
[MoviePy] >>>> Building video LaneFinding.mp4
[MoviePy] Writing video LaneFinding.mp4


  0%|          | 0/1261 [00:00<?, ?it/s]

(720, 1280)


  0%|          | 1/1261 [00:00<04:55,  4.26it/s]

(1280,)
640
(271, 640) (431, 720)
(277, 560) (437, 640)
(276, 480) (436, 560)
(269, 400) (429, 480)
(265, 320) (425, 400)
(257, 240) (417, 320)
(246, 160) (406, 240)
(237, 80) (397, 160)
(224, 0) (384, 80)
(720, 1280)


...

100%|█████████▉| 1259/1261 [04:45<00:00,  4.37it/s]

(720, 1280)
(1280,)
640
(285, 640) (445, 720)
(296, 560) (456, 640)
(294, 480) (454, 560)
(291, 400) (451, 480)
(290, 320) (450, 400)
(290, 240) (450, 320)
(288, 160) (448, 240)
(285, 80) (445, 160)
(281, 0) (441, 80)


100%|█████████▉| 1260/1261 [04:45<00:00,  4.38it/s]

(720, 1280)
(1280,)
640
(283, 640) (443, 720)
(293, 560) (453, 640)
(291, 480) (451, 560)
(289, 400) (449, 480)
(287, 320) (447, 400)
(288, 240) (448, 320)
(285, 160) (445, 240)
(282, 80) (442, 160)
(276, 0) (436, 80)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
